In [1]:
import pymorphy2
from nltk.corpus import stopwords

In [2]:
# nltk.download('averaged_perceptron_tagger_ru')

In [3]:
UNITS = ['килограмм', 'грамм', 'литр', 'пакет', 'пакетик', 'бутылка', 'упаковка', 'булка', 'пачка']
# 'буханка', 

In [4]:
stop_words = stopwords.words("russian")
stop_words.extend(['добавь', 'список', 'продуктов', 'продукты', 'продукт', 'другую', 'ещё', 'другая', 'купить'])

In [5]:
# text = 'Добавь в список продуктов молоко, сметану, 3 булки хлеба, 2 пакета сока и 3 кг картошки, ещё салями, 2 литра воды, петрушку и другую зелень, литр кока-колы'
# pure_text = '1 пачку перца сметану 3 пакетика петрушки и картошку кока-колу'
# word_pos = {word:i for i, word in enumerate(text.split())}
# word_pos

In [6]:
tokens = ["добавь", "пожалуйста", "в", "список", "покупок", "1", "пакет", "молока", "картошку", "и", "килограмм", "рыбы", "2", "фанты", "и", "1", "спрайт", "а", "ещё", "салями"]

In [7]:
def gramma_info(tokens, morph, intent_start, intent_end, remove_stopwords=True):
    gramma_info = {}

    for i in range(intent_start, intent_end):
        if tokens[i] not in stop_words:
            p = morph.parse(tokens[i])[0]
            if p.normal_form in UNITS:
                gramma_info[p.normal_form] = 'UNITS'
            elif p.normal_form.isnumeric():
                gramma_info[p.normal_form] = 'NUM'
            else:
                gramma_info[p.normal_form] = str(p.tag.POS)

    return gramma_info

In [8]:
gr_i = gramma_info(tokens, pymorphy2.MorphAnalyzer(), 5, 20, remove_stopwords=True)
gr_i

{'1': 'NUM',
 'пакет': 'UNITS',
 'молоко': 'NOUN',
 'картошка': 'NOUN',
 'килограмм': 'UNITS',
 'рыба': 'NOUN',
 '2': 'NUM',
 'фант': 'NOUN',
 'спрайт': 'NOUN',
 'салями': 'NOUN'}

In [9]:
product = ''
products = []
quantities = []
units = []
no_quantity, no_unit, no_adj = True, True, True

for item, pos in gr_i.items():
    if pos == 'NUM':
        quantities.append(int(item))
        no_quantity = False
    if pos == 'UNITS':
        units.append(item)
        no_unit = False
    if pos == 'ADJF':
        product = item + ' '
        no_adj = False
    if pos == 'NOUN':
        product += item
        products.append(product)
        product = ''
        if no_quantity:
            quantities.append(1)
        if no_unit:
            units.append(None)
        no_quantity, no_unit, no_adj = True, True, True

In [10]:
products, quantities, units

(['молоко', 'картошка', 'рыба', 'фант', 'спрайт', 'салями'],
 [1, 1, 1, 2, 1, 1],
 ['пакет', None, 'килограмм', None, None, None])